# Flight Data Exploratory Analysis

This notebook provides an interactive exploration of airline route and fare data. The dataset contains information about 638 flight routes including fares, distances, passenger counts, and market characteristics.

## Table of Contents
1. [Data Loading and Overview](#data-loading)
2. [Data Quality Assessment](#data-quality)
3. [Fare Analysis](#fare-analysis)
4. [Route Geography](#route-analysis)
5. [Market Competition](#market-analysis)
6. [Statistical Insights](#correlation-analysis)
7. [Key Findings](#summary)

In [ ]:
# Import required libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

# Set up plotting style
plt.style.use('default')
sns.set_palette("husl")
plt.rcParams['figure.figsize'] = (12, 6)

print("Libraries loaded successfully! 📊")

## 1. Data Loading and Overview {#data-loading}

In [ ]:
# Load the flight data
df = pd.read_csv('Airfares.csv')

print(f"Dataset loaded: {df.shape[0]} rows, {df.shape[1]} columns")
print(f"Memory usage: {df.memory_usage(deep=True).sum() / 1024:.2f} KB")

# Display basic information
print("\nColumn Information:")
print("-" * 50)
info_df = pd.DataFrame({
    'Column': df.columns,
    'Data Type': df.dtypes,
    'Non-Null Count': df.count(),
    'Null Count': df.isnull().sum()
})
print(info_df.to_string(index=False))

In [ ]:
# Display first few rows
print("First 5 rows of the dataset:")
df.head()

In [ ]:
# Column definitions for reference
column_definitions = {
    'S_CODE': 'Starting airport code',
    'S_CITY': 'Starting city',
    'E_CODE': 'Ending airport code', 
    'E_CITY': 'Ending city',
    'COUPON': 'Coupon fare type',
    'NEW': 'New route indicator',
    'VACATION': 'Vacation route (Yes/No)',
    'SW': 'Southwest Airlines presence (Yes/No)',
    'HI': 'Market concentration index',
    'S_INCOME': 'Starting city income level',
    'E_INCOME': 'Ending city income level',
    'S_POP': 'Starting city population',
    'E_POP': 'Ending city population',
    'SLOT': 'Airport slot restrictions',
    'GATE': 'Gate restrictions',
    'DISTANCE': 'Route distance in miles',
    'PAX': 'Number of passengers',
    'FARE': 'Ticket fare in dollars'
}

print("Column Definitions:")
print("=" * 40)
for col, definition in column_definitions.items():
    print(f"{col:<12}: {definition}")

## 2. Data Quality Assessment {#data-quality}

In [ ]:
# Check for missing values and duplicates
print("Data Quality Check:")
print("=" * 30)

missing_data = df.isnull().sum()
print(f"Missing values: {missing_data.sum()} total")

duplicates = df.duplicated().sum()
print(f"Duplicate rows: {duplicates}")

# Check unique values in categorical columns
print("\nUnique values in categorical columns:")
categorical_cols = ['VACATION', 'SW', 'SLOT', 'GATE']
for col in categorical_cols:
    unique_vals = df[col].unique()
    print(f"{col}: {list(unique_vals)}")

In [ ]:
# Basic descriptive statistics
print("Descriptive Statistics for Numerical Variables:")
numerical_cols = df.select_dtypes(include=[np.number]).columns
df[numerical_cols].describe().round(2)

## 3. Fare Analysis {#fare-analysis}

In [ ]:
# Comprehensive fare analysis
fig, axes = plt.subplots(2, 2, figsize=(16, 12))
fig.suptitle('Flight Fare Analysis', fontsize=16, fontweight='bold')

# 1. Fare distribution
axes[0, 0].hist(df['FARE'], bins=30, alpha=0.7, color='skyblue', edgecolor='black')
axes[0, 0].set_title('Distribution of Fares')
axes[0, 0].set_xlabel('Fare ($)')
axes[0, 0].set_ylabel('Frequency')
axes[0, 0].axvline(df['FARE'].mean(), color='red', linestyle='--', 
                  label=f'Mean: ${df["FARE"].mean():.2f}')
axes[0, 0].legend()

# 2. Fare vs Distance
axes[0, 1].scatter(df['DISTANCE'], df['FARE'], alpha=0.6, color='green')
axes[0, 1].set_title('Fare vs Distance')
axes[0, 1].set_xlabel('Distance (miles)')
axes[0, 1].set_ylabel('Fare ($)')

# Add trend line
z = np.polyfit(df['DISTANCE'], df['FARE'], 1)
p = np.poly1d(z)
axes[0, 1].plot(df['DISTANCE'], p(df['DISTANCE']), "r--", alpha=0.8)

# 3. Fare by Southwest presence
sw_fare = df.groupby('SW')['FARE'].mean()
axes[1, 0].bar(sw_fare.index, sw_fare.values, color=['orange', 'purple'])
axes[1, 0].set_title('Average Fare by Southwest Presence')
axes[1, 0].set_xlabel('Southwest Airlines Present')
axes[1, 0].set_ylabel('Average Fare ($)')

# 4. Fare by vacation routes
vacation_fare = df.groupby('VACATION')['FARE'].mean()
axes[1, 1].bar(vacation_fare.index, vacation_fare.values, color=['coral', 'lightblue'])
axes[1, 1].set_title('Average Fare by Route Type')
axes[1, 1].set_xlabel('Vacation Route')
axes[1, 1].set_ylabel('Average Fare ($)')

plt.tight_layout()
plt.show()

# Print insights
correlation = df['FARE'].corr(df['DISTANCE'])
print(f"\nKey Fare Insights:")
print(f"• Fare-Distance correlation: {correlation:.3f}")
print(f"• Average fare: ${df['FARE'].mean():.2f}")
print(f"• Fare range: ${df['FARE'].min():.2f} - ${df['FARE'].max():.2f}")
print(f"• Standard deviation: ${df['FARE'].std():.2f}")

In [ ]:
# Southwest Airlines impact analysis
print("Southwest Airlines Impact Analysis:")
print("=" * 40)

sw_stats = df.groupby('SW').agg({
    'FARE': ['count', 'mean', 'median', 'std'],
    'DISTANCE': 'mean',
    'PAX': 'mean'
}).round(2)

print(sw_stats)

# Calculate fare difference
sw_yes_fare = df[df['SW'] == 'Yes']['FARE'].mean()
sw_no_fare = df[df['SW'] == 'No']['FARE'].mean()
fare_reduction = ((sw_no_fare - sw_yes_fare) / sw_no_fare) * 100

print(f"\n• Southwest Airlines average fare: ${sw_yes_fare:.2f}")
print(f"• Non-Southwest average fare: ${sw_no_fare:.2f}")
print(f"• Southwest Airlines provides {fare_reduction:.1f}% lower fares")

## 4. Route Geography Analysis {#route-analysis}

In [ ]:
# Route analysis
print("Top Starting Cities by Number of Routes:")
print("=" * 45)
top_start = df['S_CITY'].value_counts().head(10)
print(top_start)

print("\nTop Destination Cities by Number of Routes:")
print("=" * 47)
top_dest = df['E_CITY'].value_counts().head(10)
print(top_dest)

In [ ]:
# Visualize route patterns
fig, axes = plt.subplots(2, 2, figsize=(16, 12))
fig.suptitle('Route Analysis', fontsize=16, fontweight='bold')

# 1. Top starting cities
top_start.head(8).plot(kind='barh', ax=axes[0, 0], color='lightgreen')
axes[0, 0].set_title('Top Starting Cities')
axes[0, 0].set_xlabel('Number of Routes')

# 2. Top destination cities
top_dest.head(8).plot(kind='barh', ax=axes[0, 1], color='lightcoral')
axes[0, 1].set_title('Top Destination Cities')
axes[0, 1].set_xlabel('Number of Routes')

# 3. Distance distribution
axes[1, 0].hist(df['DISTANCE'], bins=25, alpha=0.7, color='gold', edgecolor='black')
axes[1, 0].set_title('Distribution of Route Distances')
axes[1, 0].set_xlabel('Distance (miles)')
axes[1, 0].set_ylabel('Frequency')

# 4. Passengers vs Distance
axes[1, 1].scatter(df['DISTANCE'], df['PAX'], alpha=0.6, color='purple')
axes[1, 1].set_title('Passengers vs Distance')
axes[1, 1].set_xlabel('Distance (miles)')
axes[1, 1].set_ylabel('Passengers')

plt.tight_layout()
plt.show()

In [ ]:
# Distance and passenger statistics
print("Route Distance Statistics:")
print("=" * 30)
print(f"• Shortest route: {df['DISTANCE'].min()} miles")
print(f"• Longest route: {df['DISTANCE'].max()} miles")
print(f"• Average distance: {df['DISTANCE'].mean():.0f} miles")
print(f"• Median distance: {df['DISTANCE'].median():.0f} miles")

print("\nPassenger Statistics:")
print("=" * 25)
print(f"• Average passengers per route: {df['PAX'].mean():.0f}")
print(f"• Highest traffic route: {df['PAX'].max():,} passengers")
print(f"• Lowest traffic route: {df['PAX'].min():,} passengers")

## 5. Market Competition Analysis {#market-analysis}

In [ ]:
# Market characteristics analysis
fig, axes = plt.subplots(2, 2, figsize=(16, 12))
fig.suptitle('Market Analysis', fontsize=16, fontweight='bold')

# 1. Slot restrictions
slot_counts = df['SLOT'].value_counts()
slot_counts.plot(kind='pie', ax=axes[0, 0], autopct='%1.1f%%', colors=['lightblue', 'orange'])
axes[0, 0].set_title('Airport Slot Restrictions')
axes[0, 0].set_ylabel('')

# 2. Income correlation with fare
axes[0, 1].scatter(df['S_INCOME'], df['FARE'], alpha=0.6, color='green', label='Starting City')
axes[0, 1].scatter(df['E_INCOME'], df['FARE'], alpha=0.6, color='red', label='Destination City')
axes[0, 1].set_title('City Income vs Fare')
axes[0, 1].set_xlabel('City Income ($)')
axes[0, 1].set_ylabel('Fare ($)')
axes[0, 1].legend()

# 3. Population vs Passengers
axes[1, 0].scatter(df['S_POP'], df['PAX'], alpha=0.6, color='blue')
axes[1, 0].set_title('Starting City Population vs Passengers')
axes[1, 0].set_xlabel('Starting City Population')
axes[1, 0].set_ylabel('Passengers')

# 4. Market concentration distribution
axes[1, 1].hist(df['HI'], bins=20, alpha=0.7, color='purple', edgecolor='black')
axes[1, 1].set_title('Market Concentration Index (HI) Distribution')
axes[1, 1].set_xlabel('HI Index')
axes[1, 1].set_ylabel('Frequency')

plt.tight_layout()
plt.show()

In [ ]:
# Market restriction analysis
print("Airport Restrictions Analysis:")
print("=" * 35)

slot_analysis = df.groupby('SLOT')['FARE'].agg(['count', 'mean', 'std']).round(2)
gate_analysis = df.groupby('GATE')['FARE'].agg(['count', 'mean', 'std']).round(2)

print("Slot Restrictions Impact:")
print(slot_analysis)

print("\nGate Restrictions Impact:")
print(gate_analysis)

# Market concentration insights
print(f"\nMarket Concentration Insights:")
print(f"• Average HI index: {df['HI'].mean():.0f}")
print(f"• High concentration routes (HI > median): {(df['HI'] > df['HI'].median()).sum()}")
print(f"• Most concentrated market: HI = {df['HI'].max():.0f}")
print(f"• Least concentrated market: HI = {df['HI'].min():.0f}")

## 6. Statistical Insights {#correlation-analysis}

In [ ]:
# Correlation analysis
numerical_cols = df.select_dtypes(include=[np.number]).columns
correlation_matrix = df[numerical_cols].corr()

# Create correlation heatmap
plt.figure(figsize=(14, 10))
mask = np.triu(np.ones_like(correlation_matrix, dtype=bool))
sns.heatmap(correlation_matrix, mask=mask, annot=True, cmap='coolwarm', center=0,
           square=True, fmt='.2f', cbar_kws={"shrink": .8})
plt.title('Correlation Matrix of Numerical Variables', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

# Print strongest correlations with FARE
print("Strongest correlations with FARE:")
print("=" * 35)
fare_corr = correlation_matrix['FARE'].abs().sort_values(ascending=False)
for var, corr in fare_corr.items():
    if var != 'FARE' and abs(corr) > 0.1:
        direction = "positive" if correlation_matrix['FARE'][var] > 0 else "negative"
        print(f"• {var:<12}: {corr:.3f} ({direction})")

In [ ]:
# Advanced statistical analysis
print("Advanced Statistical Insights:")
print("=" * 35)

# Fare per mile analysis
df['FARE_PER_MILE'] = df['FARE'] / df['DISTANCE']
print(f"• Average fare per mile: ${df['FARE_PER_MILE'].mean():.3f}")

# High-value routes
high_fare_routes = df.nlargest(5, 'FARE')[['S_CITY', 'E_CITY', 'FARE', 'DISTANCE', 'FARE_PER_MILE']]
print("\nTop 5 Highest Fare Routes:")
print(high_fare_routes.to_string(index=False))

# Most efficient routes (high passengers, low fare)
df['PAX_PER_DOLLAR'] = df['PAX'] / df['FARE']
efficient_routes = df.nlargest(5, 'PAX_PER_DOLLAR')[['S_CITY', 'E_CITY', 'FARE', 'PAX', 'PAX_PER_DOLLAR']]
print("\nMost Passenger-Efficient Routes (passengers per dollar):")
print(efficient_routes.to_string(index=False))

## 7. Key Findings and Recommendations {#summary}

In [ ]:
# Summary insights
print("🛫 FLIGHT DATA ANALYSIS - KEY FINDINGS 🛬")
print("=" * 55)

print("\n📊 DATASET OVERVIEW:")
print(f"• Total routes analyzed: {len(df):,}")
print(f"• Unique starting cities: {df['S_CITY'].nunique()}")
print(f"• Unique destination cities: {df['E_CITY'].nunique()}")
print(f"• Average route distance: {df['DISTANCE'].mean():.0f} miles")

print("\n💰 FARE INSIGHTS:")
print(f"• Average airfare: ${df['FARE'].mean():.2f}")
print(f"• Fare range: ${df['FARE'].min():.2f} - ${df['FARE'].max():.2f}")
print(f"• Strong correlation between fare and distance (r={df['FARE'].corr(df['DISTANCE']):.3f})")

sw_impact = ((df[df['SW'] == 'No']['FARE'].mean() - df[df['SW'] == 'Yes']['FARE'].mean()) / 
             df[df['SW'] == 'No']['FARE'].mean() * 100)
print(f"• Southwest Airlines reduces fares by {sw_impact:.1f}% on average")

print("\n🏢 MARKET CHARACTERISTICS:")
sw_routes = (df['SW'] == 'Yes').sum()
vacation_routes = (df['VACATION'] == 'Yes').sum()
print(f"• Routes with Southwest Airlines: {sw_routes} ({sw_routes/len(df)*100:.1f}%)")
print(f"• Vacation routes: {vacation_routes} ({vacation_routes/len(df)*100:.1f}%)")
print(f"• Controlled slot airports: {(df['SLOT'] == 'Controlled').sum()} routes")

print("\n🎯 STRATEGIC RECOMMENDATIONS:")
print("• Focus on high-traffic routes between major metropolitan areas")
print("• Consider competitive pricing strategy similar to Southwest Airlines")
print("• Vacation routes may support premium pricing")
print("• Monitor market concentration for competitive opportunities")
print("• Distance-based pricing appears to be market standard")

print("\n📈 BUSINESS OPPORTUNITIES:")
# Find underserved high-income markets
high_income_routes = df[(df['S_INCOME'] > df['S_INCOME'].quantile(0.75)) & 
                       (df['E_INCOME'] > df['E_INCOME'].quantile(0.75))]
print(f"• High-income market routes: {len(high_income_routes)} opportunities")
print(f"• Average fare in high-income markets: ${high_income_routes['FARE'].mean():.2f}")

print("\n✅ Analysis complete! All visualizations and insights generated.")